<h1><center> PPOL 5203 Data Science I: Foundations <br><br> 
<font color='grey'> Data Wrangling in Pandas<br><br>
Tiago Ventura</center> <h1> 

---

**In this Notebook:**

In this notebook, we will cover standard data wrangling methods using pandas:

- Selecting Methods.
- Filtering Methods.
- Grouping and Summarization.
- Recoding Variables.


## Setup

In [58]:
import pandas as pd
import numpy as np

# set some options for pandas
pd.set_option('display.max_rows', 10)

## Data Wrangling in `pandas`


Since you are also learning R throughout DSPP, let's provide you with an overview of the main Data Wrangling Functions in R using Tidyverse and Python using `pandas`. 

**<center>Main (tidy) Data Wrangling Functions </center>**

|   [`pandas`](https://pandas.pydata.org/)      |   [`dplyr`](https://dplyr.tidyverse.org/)$^\dagger$      |     Description     |
|:---------------:|:-------------:|:-----------------------------|
| `.filter()`     | `select()`    | select column variables/index |
| `.drop()`       | `select()`    | drop selected column variables/index |
| `.rename()`     | `rename()`    | rename column variables/index |
| `.query()`      | `filter()`    | row-wise subset of a data frame by a values of a column variable/index |
| `.assign()`     |`mutate()`    | Create a new variable on the existing data frame |
| `.sort_values()`| `arrange()`   | Arrange all data values along a specified (set of) column variable(s)/indices |
| `.groupby()`    |  `group_by()`  | Index data frame by specific (set of) column variable(s)/index value(s)|
| `.agg()`        |  `summarize()` | aggregate data by specific function rules |
| `.pivot_table()`        | `spread()` | cast the data from a "long" to a "wide" format |
| `pd.melt()`        | `gather()` | cast the data from a "wide" to a "long" format |
| `.()`            | `%>%`          | piping, fluid programming, or the passing one function output to the next |


If you want to fully embrace the tidyverse style from R in Python, you should check the [`dfply` module](https://github.com/kieferk/dfply). This modules ofers an alternative to data wrangling in Python, and mirrors the popular tidyverse functionalities from R. 

We will not cover `dfply` in class because I believe you should dominate `pandas` as data scientists that are fluent in Python. However, feel free to learn and even use in your homeworks and assignment. 

In [2]:
# load worldcup dataset
wc = pd.read_csv("WorldCups.csv")
wc_matches = pd.read_csv("WorldCupMatches.csv")

In [3]:
# see wc data
wc.head()

Year      Country      Winner      Runners-Up    Third      Fourth  \
0  1930      Uruguay     Uruguay       Argentina      USA  Yugoslavia   
1  1934        Italy       Italy  Czechoslovakia  Germany     Austria   
2  1938       France       Italy         Hungary   Brazil      Sweden   
3  1950       Brazil     Uruguay          Brazil   Sweden       Spain   
4  1954  Switzerland  Germany FR         Hungary  Austria     Uruguay   

   GoalsScored  QualifiedTeams  MatchesPlayed Attendance  
0           70              13             18    590.549  
1           70              16             17    363.000  
2           84              15             18    375.700  
3           88              13             22  1.045.246  
4          140              16             26    768.607

In [4]:
# see matched data
wc_matches.head()

Year              Datetime    Stage         Stadium         City  \
0  1930  13 Jul 1930 - 15:00   Group 1         Pocitos  Montevideo    
1  1930  13 Jul 1930 - 15:00   Group 4  Parque Central  Montevideo    
2  1930  14 Jul 1930 - 12:45   Group 2  Parque Central  Montevideo    
3  1930  14 Jul 1930 - 14:50   Group 3         Pocitos  Montevideo    
4  1930  15 Jul 1930 - 16:00   Group 1  Parque Central  Montevideo    

  Home Team Name  Home Team Goals  Away Team Goals Away Team Name  \
0         France                4                1         Mexico   
1            USA                3                0        Belgium   
2     Yugoslavia                2                1         Brazil   
3        Romania                3                1           Peru   
4      Argentina                1                0         France   

  Win conditions  Attendance  Half-time Home Goals  Half-time Away Goals  \
0                     4444.0                     3                     0   
1                    18346.0                     2                     0   
2                    24059.0                     2                     0   
3                     2549.0                     1                     0   
4                    23409.0                     0                     0   

                  Referee               Assistant 1  \
0  LOMBARDI Domingo (URU)     CRISTOPHE Henry (BEL)   
1       MACIAS Jose (ARG)  MATEUCCI Francisco (URU)   
2     TEJADA Anibal (URU)   VALLARINO Ricardo (URU)   
3   WARNKEN Alberto (CHI)       LANGENUS Jean (BEL)   
4     REGO Gilberto (BRA)      SAUCEDO Ulises (BOL)   

                  Assistant 2  RoundID  MatchID Home Team Initials  \
0         REGO Gilberto (BRA)      201     1096                FRA   
1       WARNKEN Alberto (CHI)      201     1090                USA   
2         BALWAY Thomas (FRA)      201     1093                YUG   
3    MATEUCCI Francisco (URU)      201     1098                ROU   
4  RADULESCU Constantin (ROU)      201     1085                ARG   

  Away Team Initials  
0                MEX  
1                BEL  
2                BRA  
3                PER  
4                FRA

## Piping

If you started your data science career, or are simultaneously learning, R, it is likely you were exposed to the `tidyverse` world, and the famous `pipe`. Here it is for you:

![](https://magrittr.tidyverse.org/logo.png)


Pipe operators, introduced by tha magritt package in R, absolutely transformed the way of writing coding in R. It did for good reasons. And in the past few years, piping has also become more prevalent in Python. 

Pipe operators allows you to:

- **Chain together data manipulations in a single operational sequence.**

In pandas, piping allows you to chain together in a single workflow a sequence of methods. Pipe operators (`.pipe()`) in `pandas` were introduced relatively recently (version 0.16.2).

Let's see some examples using the world cup data. We want to count which country played more world cup games.

In [56]:
# read world cup data
wc = pd.read_csv("WorldCupMatches.csv")

#### _Method 1_: sequentially overwrite the object

In [59]:
wc = pd.read_csv("WorldCupMatches.csv")

# select columns
wc = wc.filter(['Year','Home Team Name','Away Team Name'])

# make it tidy
wc = wc.melt(id_vars=["Year"], var_name="var", value_name="country")

# group by
wc = wc.groupby("country")

# count occurrences
wc = wc.size()

# move index to column with new name
wc = wc.reset_index(name="n")

# sort
wc = wc.sort_values(by="n", ascending=False)

# print 10
wc.head(10)

country    n
7        Brazil  108
39        Italy   83
2     Argentina   81
25      England   62
29   Germany FR   62
26       France   61
66        Spain   59
45       Mexico   54
47  Netherlands   54
74      Uruguay   52

#### Method 2: Pandas Pipe

In [67]:
wc = pd.read_csv("WorldCupMatches.csv")

# select columns
wc_10 = ( # encapsulate
            wc.filter(['Year','Home Team Name','Away Team Name']).
             melt(id_vars=["Year"], var_name="var", value_name="country"). 
             groupby("country"). 
             size(). 
             reset_index(name="n").
             sort_values(by="n", ascending=False)
        ) # close

# print 10
wc_10.head(10)

country    n
7        Brazil  108
39        Italy   83
2     Argentina   81
25      England   62
29   Germany FR   62
26       France   61
66        Spain   59
45       Mexico   54
47  Netherlands   54
74      Uruguay   52

#### Final notes in Piping

<div class="alert alert-block alert-info">

To understand pipes, you should always remember: **data in, data out**. That's what pipes do: apply methods sequentially to `pandas dataframes`. 
    
</div>


It should be clear by now, but these are some of the advantages of piping your code: 
    
- Improves code structure
- Eliminates intermediate variables	
- Can improve code readability
- Memory efficiency by eliminating intermediate variables
- Makes easies to add steps anywhere in the sequence of operations

Keep in mind pipes also have some disadvantages. In my view, specially when working in notebook in which you do not execute line by line, pipes can make codes a bit harder to debug and also embed errors that could be more easily perceived by examining intermediate variables. 

###  Column-Wise Operations

For data wrangling tasks at the columns of your data frame, we will discuss: 

- Select columns
- Drop  columns
- Create new columns
- Rename columns


### For all operations we will see index vs function-based implementation

### Select Columns

**Functionality:**

- Select specific variables/column indices

**Implementation**

- Traditional indexing in pandas
- `.loc()` methods
- `pd.filter()` methods (allows piping). 


#### Select via index

In [73]:
# load worldcup dataset
wc = pd.read_csv("WorldCups.csv")
wc_matches = pd.read_csv("WorldCupMatches.csv")

In [74]:
## simple index for columns
wc["Year"]

0     1930
1     1934
2     1938
3     1950
4     1954
      ... 
15    1998
16    2002
17    2006
18    2010
19    2014
Name: Year, Length: 20, dtype: int64

In [75]:
## using dot method
wc.Year

0     1930
1     1934
2     1938
3     1950
4     1954
      ... 
15    1998
16    2002
17    2006
18    2010
19    2014
Name: Year, Length: 20, dtype: int64

In [76]:
## using .loc methods
wc.loc[:,"Year"]

0     1930
1     1934
2     1938
3     1950
4     1954
      ... 
15    1998
16    2002
17    2006
18    2010
19    2014
Name: Year, Length: 20, dtype: int64

Notice, in all cases, the output comes as a `pandas series`. If you would like the output to be a full data frame, or if you need to select multiple columns, you should give a list of indexes as inputs

In [77]:
wc[["Winner", "Year"]]

Winner  Year
0      Uruguay  1930
1        Italy  1934
2        Italy  1938
3      Uruguay  1950
4   Germany FR  1954
..         ...   ...
15      France  1998
16      Brazil  2002
17       Italy  2006
18       Spain  2010
19     Germany  2014

[20 rows x 2 columns]

#### `.loc()` methods for selecting columns

It allows you to select multible variables in between column names!

In [80]:
# .loc for a single or multiple columns
wc.loc[0:10, "Year":"Winner"]

Year      Country      Winner
0   1930      Uruguay     Uruguay
1   1934        Italy       Italy
2   1938       France       Italy
3   1950       Brazil     Uruguay
4   1954  Switzerland  Germany FR
..   ...          ...         ...
6   1962        Chile      Brazil
7   1966      England     England
8   1970       Mexico      Brazil
9   1974      Germany  Germany FR
10  1978    Argentina   Argentina

[11 rows x 3 columns]

In [82]:
# iloc for numeric positions
wc.iloc[0:10,0:3]

Year      Country      Winner
0  1930      Uruguay     Uruguay
1  1934        Italy       Italy
2  1938       France       Italy
3  1950       Brazil     Uruguay
4  1954  Switzerland  Germany FR
5  1958       Sweden      Brazil
6  1962        Chile      Brazil
7  1966      England     England
8  1970       Mexico      Brazil
9  1974      Germany  Germany FR

#### `.filter()` method

The `filter` methods in Pandas works similarly to the `select` function in the Tidyverse in R. It has the following advantages: 

- Allows for a piping approach
- Can be combined with regex queries for selectins columns

In [83]:
# simple filter
wc.filter(["Year", "Winner"])

Year      Winner
0   1930     Uruguay
1   1934       Italy
2   1938       Italy
3   1950     Uruguay
4   1954  Germany FR
..   ...         ...
15  1998      France
16  2002      Brazil
17  2006       Italy
18  2010       Spain
19  2014     Germany

[20 rows x 2 columns]

**Using the like parameter:** Select columns that contain a specific substring.

In [84]:
# like parameter. In R: data %>% select(contains("Away"))
wc_matches.filter(like="Away")

Away Team Goals Away Team Name  Half-time Away Goals Away Team Initials
0                  1         Mexico                     0                MEX
1                  0        Belgium                     0                BEL
2                  1         Brazil                     0                BRA
3                  1           Peru                     0                PER
4                  0         France                     0                FRA
..               ...            ...                   ...                ...
847                0     Costa Rica                     0                CRC
848                7        Germany                     5                GER
849                0      Argentina                     0                ARG
850                3    Netherlands                     2                NED
851                0      Argentina                     0                ARG

[852 rows x 4 columns]

**Using the Regex:** You can also input regex queries for selecting columns. More and More flexibility

In [85]:
# starts with
wc_matches.filter(regex="^Away")

Away Team Goals Away Team Name Away Team Initials
0                  1         Mexico                MEX
1                  0        Belgium                BEL
2                  1         Brazil                BRA
3                  1           Peru                PER
4                  0         France                FRA
..               ...            ...                ...
847                0     Costa Rica                CRC
848                7        Germany                GER
849                0      Argentina                ARG
850                3    Netherlands                NED
851                0      Argentina                ARG

[852 rows x 3 columns]

In [86]:
# ends with
wc_matches.filter(regex="Initials$")


Home Team Initials Away Team Initials
0                  FRA                MEX
1                  USA                BEL
2                  YUG                BRA
3                  ROU                PER
4                  ARG                FRA
..                 ...                ...
847                NED                CRC
848                BRA                GER
849                NED                ARG
850                BRA                NED
851                GER                ARG

[852 rows x 2 columns]

### Drop Columns

**Functionality:**

Drop specific variables/column indices

**Implementation:**

- Indexing + Boolean operations
- .loc() methods
- pd.drop() methods (allows piping).

In [59]:
# Indexing: Bit too much?!
# .isin returns a boolean.
wc[wc.columns[~wc.columns.isin(["Year", "Country"])]]

Winner      Runners-Up    Third      Fourth  GoalsScored  \
0     Uruguay       Argentina      USA  Yugoslavia           70   
1       Italy  Czechoslovakia  Germany     Austria           70   
2       Italy         Hungary   Brazil      Sweden           84   
3     Uruguay          Brazil   Sweden       Spain           88   
4  Germany FR         Hungary  Austria     Uruguay          140   

   QualifiedTeams  MatchesPlayed Attendance  
0              13             18    590.549  
1              16             17    363.000  
2              15             18    375.700  
3              13             22  1.045.246  
4              16             26    768.607

In [90]:
# What is going on here?
~wc.columns.isin(["Year", "Country"])

array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True])


<div class="alert alert-block alert-warning">

bol_col is an array of booleans. If you throw them directly as a index,  Pandas interprets `wc[bol_col]` as trying to index the rows of the DataFrame, not the columns. This is because when you pass a boolean array directly to the DataFrame indexing operator [], pandas assumes you're trying to index rows based on the boolean array.
</div>    

In [88]:
# loc methods
wc.loc[:,~wc.columns.isin(["Year", "Country"])]

Winner      Runners-Up        Third          Fourth  GoalsScored  \
0      Uruguay       Argentina          USA      Yugoslavia           70   
1        Italy  Czechoslovakia      Germany         Austria           70   
2        Italy         Hungary       Brazil          Sweden           84   
3      Uruguay          Brazil       Sweden           Spain           88   
4   Germany FR         Hungary      Austria         Uruguay          140   
..         ...             ...          ...             ...          ...   
15      France          Brazil      Croatia     Netherlands          171   
16      Brazil         Germany       Turkey  Korea Republic          161   
17       Italy          France      Germany        Portugal          147   
18       Spain     Netherlands      Germany         Uruguay          145   
19     Germany       Argentina  Netherlands          Brazil          171   

    QualifiedTeams  MatchesPlayed Attendance  
0               13             18    590.549  
1               16             17    363.000  
2               15             18    375.700  
3               13             22  1.045.246  
4               16             26    768.607  
..             ...            ...        ...  
15              32             64  2.785.100  
16              32             64  2.705.197  
17              32             64  3.359.439  
18              32             64  3.178.856  
19              32             64  3.386.810  

[20 rows x 8 columns]

In [91]:
# indexing -> error
bol_col = ~wc.columns.isin(["Year", "Country"])
wc[bol_col]

ValueError: Item wrong length 10 instead of 20.

In [92]:
# With .loc, it works
wc.loc[:,bol_col]

Winner      Runners-Up        Third          Fourth  GoalsScored  \
0      Uruguay       Argentina          USA      Yugoslavia           70   
1        Italy  Czechoslovakia      Germany         Austria           70   
2        Italy         Hungary       Brazil          Sweden           84   
3      Uruguay          Brazil       Sweden           Spain           88   
4   Germany FR         Hungary      Austria         Uruguay          140   
..         ...             ...          ...             ...          ...   
15      France          Brazil      Croatia     Netherlands          171   
16      Brazil         Germany       Turkey  Korea Republic          161   
17       Italy          France      Germany        Portugal          147   
18       Spain     Netherlands      Germany         Uruguay          145   
19     Germany       Argentina  Netherlands          Brazil          171   

    QualifiedTeams  MatchesPlayed Attendance  
0               13             18    590.549  
1               16             17    363.000  
2               15             18    375.700  
3               13             22  1.045.246  
4               16             26    768.607  
..             ...            ...        ...  
15              32             64  2.785.100  
16              32             64  2.705.197  
17              32             64  3.359.439  
18              32             64  3.178.856  
19              32             64  3.386.810  

[20 rows x 8 columns]

#### `pd.drop()` methods: easier way to go

In [96]:
wc.drop(columns=["Year"])

Country      Winner      Runners-Up        Third          Fourth  \
0        Uruguay     Uruguay       Argentina          USA      Yugoslavia   
1          Italy       Italy  Czechoslovakia      Germany         Austria   
2         France       Italy         Hungary       Brazil          Sweden   
3         Brazil     Uruguay          Brazil       Sweden           Spain   
4    Switzerland  Germany FR         Hungary      Austria         Uruguay   
..           ...         ...             ...          ...             ...   
15        France      France          Brazil      Croatia     Netherlands   
16   Korea/Japan      Brazil         Germany       Turkey  Korea Republic   
17       Germany       Italy          France      Germany        Portugal   
18  South Africa       Spain     Netherlands      Germany         Uruguay   
19        Brazil     Germany       Argentina  Netherlands          Brazil   

    GoalsScored  QualifiedTeams  MatchesPlayed Attendance  
0            70              13             18    590.549  
1            70              16             17    363.000  
2            84              15             18    375.700  
3            88              13             22  1.045.246  
4           140              16             26    768.607  
..          ...             ...            ...        ...  
15          171              32             64  2.785.100  
16          161              32             64  2.705.197  
17          147              32             64  3.359.439  
18          145              32             64  3.178.856  
19          171              32             64  3.386.810  

[20 rows x 9 columns]

In [97]:
help(pd.DataFrame.drop)

Help on function drop in module pandas.core.frame:

drop(self, labels: 'IndexLabel' = None, *, axis: 'Axis' = 0, index: 'IndexLabel' = None, columns: 'IndexLabel' = None, level: 'Level' = None, inplace: 'bool' = False, errors: 'IgnoreRaise' = 'raise') -> 'DataFrame | None'
    Drop specified labels from rows or columns.
    
    Remove rows or columns by specifying label names and corresponding
    axis, or by specifying directly index or column names. When using a
    multi-index, labels on different levels can be removed by specifying
    the level. See the `user guide <advanced.shown_levels>`
    for more information about the now unused levels.
    
    Parameters
    ----------
    labels : single label or list-like
        Index or column labels to drop. A tuple will be used as a single
        label and not treated as a list-like.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Whether to drop labels from the index (0 or 'index') or
        columns (1 or 'columns').

### Create new columns

**Functionality:**

- Create a new column/index given inputs and or transformations from other columns. 

**Implementation**

Traditional index assignment. Advantages:
  
+ Looks like a dictionary operation
+ Overwrites the data frame


`.assign()` method. Advantage:
    
+ It returns a dataframe so you can chain/pipe operations
+ Can create multiple variables in a single call
+ Easy to combine with numpy + lambda functions
+ Improves readibility.
    
Let's see examples of both methods:


#### Transformation via index assignment

In [98]:
# With built in math operations
wc["av_goals_matches"] = wc["GoalsScored"]/wc["MatchesPlayed"]
wc.head()

Year      Country      Winner      Runners-Up    Third      Fourth  \
0  1930      Uruguay     Uruguay       Argentina      USA  Yugoslavia   
1  1934        Italy       Italy  Czechoslovakia  Germany     Austria   
2  1938       France       Italy         Hungary   Brazil      Sweden   
3  1950       Brazil     Uruguay          Brazil   Sweden       Spain   
4  1954  Switzerland  Germany FR         Hungary  Austria     Uruguay   

   GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  
0           70              13             18    590.549          3.888889  
1           70              16             17    363.000          4.117647  
2           84              15             18    375.700          4.666667  
3           88              13             22  1.045.246          4.000000  
4          140              16             26    768.607          5.384615

In [99]:
## with numpy
wc["winner_and_hoster"] = np.where(wc.Country==wc.Winner, True, False)
wc[["Year", "Winner", "Country", "winner_and_hoster"]].head(5)

Year      Winner      Country  winner_and_hoster
0  1930     Uruguay      Uruguay               True
1  1934       Italy        Italy               True
2  1938       Italy       France              False
3  1950     Uruguay       Brazil              False
4  1954  Germany FR  Switzerland              False

#### Transformation via apply methods by rows


In [101]:
# with an apply method + function
wc["av_goals_matches"] = wc.apply(lambda x: x["GoalsScored"]/x["MatchesPlayed"], axis=1)
wc.head()

Year      Country      Winner      Runners-Up    Third      Fourth  \
0  1930      Uruguay     Uruguay       Argentina      USA  Yugoslavia   
1  1934        Italy       Italy  Czechoslovakia  Germany     Austria   
2  1938       France       Italy         Hungary   Brazil      Sweden   
3  1950       Brazil     Uruguay          Brazil   Sweden       Spain   
4  1954  Switzerland  Germany FR         Hungary  Austria     Uruguay   

   GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
0           70              13             18    590.549          3.888889   
1           70              16             17    363.000          4.117647   
2           84              15             18    375.700          4.666667   
3           88              13             22  1.045.246          4.000000   
4          140              16             26    768.607          5.384615   

   winner_and_hoster  
0               True  
1               True  
2              False  
3              False  
4              False

#### `.assign()` method.

Allows you to create variables with methods chaining. 

In [102]:
## when the winner was also hosting the world cup?
(wc.
 # multiple variables
 assign(final=wc.Winner + " vs " + wc['Runners-Up'],
        winner_and_hoster_np = np.where(wc.Winner==wc.Country, True, False), 
        av_goals_matches = lambda x: x["GoalsScored"]/x["MatchesPlayed"], 
        ).
  #allows for methods chaining
 filter(["Year", "Winner", "Country", "final", "winner_and_hoster_np", "av_goals_matches"]).
 head(5))

Year      Winner      Country                    final  \
0  1930     Uruguay      Uruguay     Uruguay vs Argentina   
1  1934       Italy        Italy  Italy vs Czechoslovakia   
2  1938       Italy       France         Italy vs Hungary   
3  1950     Uruguay       Brazil        Uruguay vs Brazil   
4  1954  Germany FR  Switzerland    Germany FR vs Hungary   

   winner_and_hoster_np  av_goals_matches  
0                  True          3.888889  
1                  True          4.117647  
2                 False          4.666667  
3                 False          4.000000  
4                 False          5.384615

`.assign` also allows for the use of newly create variables in the same chain. To do that, you need to make use of the lambda function

In [104]:
# Notice calling the recently created variable final
(wc.
 assign(final= wc.Winner + " vs " + wc['Runners-Up'],
        best_three = lambda x: x["final"] + " and " + x["Third"]).
 filter(["best_three","final", "Country"]).
 head(5)
)

best_three                    final      Country
0         Uruguay vs Argentina and USA     Uruguay vs Argentina      Uruguay
1  Italy vs Czechoslovakia and Germany  Italy vs Czechoslovakia        Italy
2          Italy vs Hungary and Brazil         Italy vs Hungary       France
3         Uruguay vs Brazil and Sweden        Uruguay vs Brazil       Brazil
4    Germany FR vs Hungary and Austria    Germany FR vs Hungary  Switzerland

<div class="alert alert-block alert-danger">

**Alert:** Spend a few seconds trying to understand the use of the lambda function in the code above. 

</div>   

The combination of lambda (or a normal function) and `.assign()` is actually a nice property that resambles the properties of `mutate` in R. A lambda function (or a normal function) with .assign passes in the **current state of the dataframe** into the function. Because we create the variable in the state before, the lambda function allows us to access this new variable, and manipulate it in sequence. This also works for cases of grouped data, or chains in which we filter observations and transform the dataframe

To make this point clear, see how doing the same operation without a lambda function will throw an error: 


In [103]:
# will throw an error
(wc.
 assign(final= wc.Winner + " vs " + wc['Runners-Up'],
        best_three = wc["final"] + " and " + wc["Third"]).
 filter(["best_three","final", "Country"])
)

KeyError: 'final'

### Renaming 

**Functionality:**

Rename a columns directly or a via a function. 

**Implementation:**

- Use dictionaries!

In [105]:
# Pandas: renaming variables using the rename method
wc.rename(columns={"Year":"year"}).head(3)

year  Country   Winner      Runners-Up    Third      Fourth  GoalsScored  \
0  1930  Uruguay  Uruguay       Argentina      USA  Yugoslavia           70   
1  1934    Italy    Italy  Czechoslovakia  Germany     Austria           70   
2  1938   France    Italy         Hungary   Brazil      Sweden           84   

   QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
0              13             18    590.549          3.888889   
1              16             17    363.000          4.117647   
2              15             18    375.700          4.666667   

   winner_and_hoster  
0               True  
1               True  
2              False

In [106]:
# we can use dictionary comprehension to apply functions in all
wc.rename(columns={col: col.lower() for col in wc.columns}).head(5)

year      country      winner      runners-up    third      fourth  \
0  1930      Uruguay     Uruguay       Argentina      USA  Yugoslavia   
1  1934        Italy       Italy  Czechoslovakia  Germany     Austria   
2  1938       France       Italy         Hungary   Brazil      Sweden   
3  1950       Brazil     Uruguay          Brazil   Sweden       Spain   
4  1954  Switzerland  Germany FR         Hungary  Austria     Uruguay   

   goalsscored  qualifiedteams  matchesplayed attendance  av_goals_matches  \
0           70              13             18    590.549          3.888889   
1           70              16             17    363.000          4.117647   
2           84              15             18    375.700          4.666667   
3           88              13             22  1.045.246          4.000000   
4          140              16             26    768.607          5.384615   

   winner_and_hoster  
0               True  
1               True  
2              False  
3              False  
4              False

In [107]:
# Or for only a set of the columns by given them as inputs
wc.rename(columns={col: col.lower() for col in ["Year", "Country"]}).head(5)

year      country      Winner      Runners-Up    Third      Fourth  \
0  1930      Uruguay     Uruguay       Argentina      USA  Yugoslavia   
1  1934        Italy       Italy  Czechoslovakia  Germany     Austria   
2  1938       France       Italy         Hungary   Brazil      Sweden   
3  1950       Brazil     Uruguay          Brazil   Sweden       Spain   
4  1954  Switzerland  Germany FR         Hungary  Austria     Uruguay   

   GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
0           70              13             18    590.549          3.888889   
1           70              16             17    363.000          4.117647   
2           84              15             18    375.700          4.666667   
3           88              13             22  1.045.246          4.000000   
4          140              16             26    768.607          5.384615   

   winner_and_hoster  
0               True  
1               True  
2              False  
3              False  
4              False

### Practice

In this practice questions, we'll hone our data manipulation skills by examining conflict event data generated by the [Armed Conflict Location & Event Data Project (ACLED)](https://acleddata.com/#/dashboard). The aim is to practice some of the data manipulation functions covered in the lecture. 

#### Data

ACLED is a "disaggregated data collection, analysis, and crisis mapping project. ACLED collects the dates, actors, locations, fatalities, and modalities of all reported political violence and protest events across Africa, South Asia, Southeast Asia, the Middle East, Central Asia and the Caucasus, Latin America and the Caribbean, and Southeastern and Eastern Europe and the Balkans." For this exercise, we'll focus just on the data pertaining to _Africa_. For more information regarding these data, please consult the [ACLED methodology](https://acleddata.com/resources/methodology/).

<a href="https://tiagoventura.github.io/ppol5203_fall_2023/lecture_notes/week-06/data_week6.zip" download><strong>Download the ACLED data</strong></a>.

- Open the Data
- Create a new dataset with  all columns the words event or actors
- Rename all columns replacing the the word event to "event_acled_south_america"
- Create three new columns: 

    - a binary representation for fatalities, 
    - a single columns combining the columns: "region",	"country",
    - a new column recoding the inter1 variable with three string labels. These labels should indicate when the iter1 variable is equal to its median, below the median and above the median values.

In [ ]:
## add your solutions here

## Row-Wise Operations

For data wrangling tasks at the rows of your data frame, we will discuss: 

- Subsetting
- Filtering distinct values
- Recoding values 
- Grouping and Summarizing
- Grouping and Transforming
- Sorting values

### Subsetting

**Functionality:**

Slice the dataframe row-wise following a certain input. 

**Implementation:**

- index based implementation
- `.loc` or `.iloc` methods
- `.query`


#### Subsetting by index

In [108]:
# index based implementation
wc[wc.Year<1990]

Year      Country      Winner      Runners-Up       Third      Fourth  \
0   1930      Uruguay     Uruguay       Argentina         USA  Yugoslavia   
1   1934        Italy       Italy  Czechoslovakia     Germany     Austria   
2   1938       France       Italy         Hungary      Brazil      Sweden   
3   1950       Brazil     Uruguay          Brazil      Sweden       Spain   
4   1954  Switzerland  Germany FR         Hungary     Austria     Uruguay   
..   ...          ...         ...             ...         ...         ...   
8   1970       Mexico      Brazil           Italy  Germany FR     Uruguay   
9   1974      Germany  Germany FR     Netherlands      Poland      Brazil   
10  1978    Argentina   Argentina     Netherlands      Brazil       Italy   
11  1982        Spain       Italy      Germany FR      Poland      France   
12  1986       Mexico   Argentina      Germany FR      France     Belgium   

    GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
0            70              13             18    590.549          3.888889   
1            70              16             17    363.000          4.117647   
2            84              15             18    375.700          4.666667   
3            88              13             22  1.045.246          4.000000   
4           140              16             26    768.607          5.384615   
..          ...             ...            ...        ...               ...   
8            95              16             32  1.603.975          2.968750   
9            97              16             38  1.865.753          2.552632   
10          102              16             38  1.545.791          2.684211   
11          146              24             52  2.109.723          2.807692   
12          132              24             52  2.394.031          2.538462   

    winner_and_hoster  
0                True  
1                True  
2               False  
3               False  
4               False  
..                ...  
8               False  
9               False  
10               True  
11              False  
12              False  

[13 rows x 12 columns]

In [109]:
# or with multiple condition
wc[(wc.Year<1990)&(wc.Year>1940)]

Year      Country      Winner      Runners-Up       Third        Fourth  \
3   1950       Brazil     Uruguay          Brazil      Sweden         Spain   
4   1954  Switzerland  Germany FR         Hungary     Austria       Uruguay   
5   1958       Sweden      Brazil          Sweden      France    Germany FR   
6   1962        Chile      Brazil  Czechoslovakia       Chile    Yugoslavia   
7   1966      England     England      Germany FR    Portugal  Soviet Union   
8   1970       Mexico      Brazil           Italy  Germany FR       Uruguay   
9   1974      Germany  Germany FR     Netherlands      Poland        Brazil   
10  1978    Argentina   Argentina     Netherlands      Brazil         Italy   
11  1982        Spain       Italy      Germany FR      Poland        France   
12  1986       Mexico   Argentina      Germany FR      France       Belgium   

    GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
3            88              13             22  1.045.246          4.000000   
4           140              16             26    768.607          5.384615   
5           126              16             35    819.810          3.600000   
6            89              16             32    893.172          2.781250   
7            89              16             32  1.563.135          2.781250   
8            95              16             32  1.603.975          2.968750   
9            97              16             38  1.865.753          2.552632   
10          102              16             38  1.545.791          2.684211   
11          146              24             52  2.109.723          2.807692   
12          132              24             52  2.394.031          2.538462   

    winner_and_hoster  
3               False  
4               False  
5               False  
6               False  
7                True  
8               False  
9               False  
10               True  
11              False  
12              False

#### Subsetting with `.loc()`  method

In [111]:
(wc.Year<1990)&(wc.Year>1940)

0     False
1     False
2     False
3      True
4      True
      ...  
15    False
16    False
17    False
18    False
19    False
Name: Year, Length: 20, dtype: bool

In [112]:
# pretty much the same
wc.loc[(wc.Year<1990)&(wc.Year>1940),:]

Year      Country      Winner      Runners-Up       Third        Fourth  \
3   1950       Brazil     Uruguay          Brazil      Sweden         Spain   
4   1954  Switzerland  Germany FR         Hungary     Austria       Uruguay   
5   1958       Sweden      Brazil          Sweden      France    Germany FR   
6   1962        Chile      Brazil  Czechoslovakia       Chile    Yugoslavia   
7   1966      England     England      Germany FR    Portugal  Soviet Union   
8   1970       Mexico      Brazil           Italy  Germany FR       Uruguay   
9   1974      Germany  Germany FR     Netherlands      Poland        Brazil   
10  1978    Argentina   Argentina     Netherlands      Brazil         Italy   
11  1982        Spain       Italy      Germany FR      Poland        France   
12  1986       Mexico   Argentina      Germany FR      France       Belgium   

    GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
3            88              13             22  1.045.246          4.000000   
4           140              16             26    768.607          5.384615   
5           126              16             35    819.810          3.600000   
6            89              16             32    893.172          2.781250   
7            89              16             32  1.563.135          2.781250   
8            95              16             32  1.603.975          2.968750   
9            97              16             38  1.865.753          2.552632   
10          102              16             38  1.545.791          2.684211   
11          146              24             52  2.109.723          2.807692   
12          132              24             52  2.394.031          2.538462   

    winner_and_hoster  
3               False  
4               False  
5               False  
6               False  
7                True  
8               False  
9               False  
10               True  
11              False  
12              False

#### Subsetting with `.query()`: a pipe approach

As before, you can use `.query()` methods to a more reable and pipeble approach. Notice the inside of the quotation marks use only the column name. 

In [113]:
wc.query('Year<1990 & Year>1940')

Year      Country      Winner      Runners-Up       Third        Fourth  \
3   1950       Brazil     Uruguay          Brazil      Sweden         Spain   
4   1954  Switzerland  Germany FR         Hungary     Austria       Uruguay   
5   1958       Sweden      Brazil          Sweden      France    Germany FR   
6   1962        Chile      Brazil  Czechoslovakia       Chile    Yugoslavia   
7   1966      England     England      Germany FR    Portugal  Soviet Union   
8   1970       Mexico      Brazil           Italy  Germany FR       Uruguay   
9   1974      Germany  Germany FR     Netherlands      Poland        Brazil   
10  1978    Argentina   Argentina     Netherlands      Brazil         Italy   
11  1982        Spain       Italy      Germany FR      Poland        France   
12  1986       Mexico   Argentina      Germany FR      France       Belgium   

    GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
3            88              13             22  1.045.246          4.000000   
4           140              16             26    768.607          5.384615   
5           126              16             35    819.810          3.600000   
6            89              16             32    893.172          2.781250   
7            89              16             32  1.563.135          2.781250   
8            95              16             32  1.603.975          2.968750   
9            97              16             38  1.865.753          2.552632   
10          102              16             38  1.545.791          2.684211   
11          146              24             52  2.109.723          2.807692   
12          132              24             52  2.394.031          2.538462   

    winner_and_hoster  
3               False  
4               False  
5               False  
6               False  
7                True  
8               False  
9               False  
10               True  
11              False  
12              False

### Other types of subsetting

#### Subset by distinct entry

In [114]:
# Pandas: drop duplicative entries for a specific variable
# notice here you are actually deleting important rows
wc.drop_duplicates("Country")

Year       Country      Winner      Runners-Up    Third          Fourth  \
0   1930       Uruguay     Uruguay       Argentina      USA      Yugoslavia   
1   1934         Italy       Italy  Czechoslovakia  Germany         Austria   
2   1938        France       Italy         Hungary   Brazil          Sweden   
3   1950        Brazil     Uruguay          Brazil   Sweden           Spain   
4   1954   Switzerland  Germany FR         Hungary  Austria         Uruguay   
..   ...           ...         ...             ...      ...             ...   
10  1978     Argentina   Argentina     Netherlands   Brazil           Italy   
11  1982         Spain       Italy      Germany FR   Poland          France   
14  1994           USA      Brazil           Italy   Sweden        Bulgaria   
16  2002   Korea/Japan      Brazil         Germany   Turkey  Korea Republic   
18  2010  South Africa       Spain     Netherlands  Germany         Uruguay   

    GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
0            70              13             18    590.549          3.888889   
1            70              16             17    363.000          4.117647   
2            84              15             18    375.700          4.666667   
3            88              13             22  1.045.246          4.000000   
4           140              16             26    768.607          5.384615   
..          ...             ...            ...        ...               ...   
10          102              16             38  1.545.791          2.684211   
11          146              24             52  2.109.723          2.807692   
14          141              24             52  3.587.538          2.711538   
16          161              32             64  2.705.197          2.515625   
18          145              32             64  3.178.856          2.265625   

    winner_and_hoster  
0                True  
1                True  
2               False  
3               False  
4               False  
..                ...  
10               True  
11              False  
14              False  
16              False  
18              False  

[15 rows x 12 columns]

### Subset by sampling

In [115]:
# Pandas: randomly sample N number of rows from the data
wc.sample(3)

Year  Country      Winner   Runners-Up   Third  Fourth  GoalsScored  \
11  1982    Spain       Italy   Germany FR  Poland  France          146   
9   1974  Germany  Germany FR  Netherlands  Poland  Brazil           97   
2   1938   France       Italy      Hungary  Brazil  Sweden           84   

    QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
11              24             52  2.109.723          2.807692   
9               16             38  1.865.753          2.552632   
2               15             18    375.700          4.666667   

    winner_and_hoster  
11              False  
9               False  
2               False

### Recoding values

**Functionality:**

Recode a value given certain conditions. This type of transformation is one of the most importants in data cleaning. 

**Implementation:**

There are many ways to recode variables in Python. We will showcase four of the most useful in my view. 

First, we will see more generalized row-wise approach in pandas using: 

- `map()`
- `filter()`

Then we will see two vectorized solutions using numpy: 

- `np.where()`
- `np.select()` 


#### Recode with map() + dictionaries

The `map()` function is used to substitute each value in a **Series** with another value. 

- It takes a series as input
- Uses a function/dictionary to transform values
- Returns a series. 

In [117]:
# map + dictionary to recode to create a dummy for certain country

# create a map function
mapping = {"Brazil":1}

# map the values
wc["brazil_winner"]= wc["Winner"].map(mapping)

# Fill missing values with a default value (e.g., 0)
wc["brazil_winner"].fillna(0, inplace=True)

# see results
wc.tail(5)

Year       Country   Winner   Runners-Up        Third          Fourth  \
15  1998        France   France       Brazil      Croatia     Netherlands   
16  2002   Korea/Japan   Brazil      Germany       Turkey  Korea Republic   
17  2006       Germany    Italy       France      Germany        Portugal   
18  2010  South Africa    Spain  Netherlands      Germany         Uruguay   
19  2014        Brazil  Germany    Argentina  Netherlands          Brazil   

    GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
15          171              32             64  2.785.100          2.671875   
16          161              32             64  2.705.197          2.515625   
17          147              32             64  3.359.439          2.296875   
18          145              32             64  3.178.856          2.265625   
19          171              32             64  3.386.810          2.671875   

    winner_and_hoster  brazil_winner  
15               True            0.0  
16              False            1.0  
17              False            0.0  
18              False            0.0  
19              False            0.0

#### Recode with apply() + function

The apply() function is used when you want to apply a function along the axis of a DataFrame (either rows or columns).  This function can be both an in-built function or a user-defined function.

In [118]:
# apply + function to recode
def get_dummies(x):
    if x =="Brazil":
        return 1
    else:
        return 0
    
# apply function  
wc['Winner'].apply(get_dummies).head(10)

0    0
1    0
2    0
3    0
4    0
5    1
6    1
7    0
8    1
9    0
Name: Winner, dtype: int64

Notice, we can make it more general by providing a argument to the function

In [121]:
# apply + function to recode
def get_dummies(x, country):
    if x == country:
        return 1
    else:
        return 0
    
# Apply function with Uruguay now
wc["winner_uruguay"] = wc['Winner'].apply(get_dummies, country="Uruguay").head(10)

Here we are using apply for a column, so we do not need to indicate axis=1. That's also why we are inputing the columns as a serie. Notice we could do the same with assign: 

In [131]:
# using apply inside of assign
wc.assign(winner_uruguay=wc['Winner'].apply(get_dummies, country="Uruguay"))

Year       Country      Winner      Runners-Up        Third  \
0   1930       Uruguay     Uruguay       Argentina          USA   
1   1934         Italy       Italy  Czechoslovakia      Germany   
2   1938        France       Italy         Hungary       Brazil   
3   1950        Brazil     Uruguay          Brazil       Sweden   
4   1954   Switzerland  Germany FR         Hungary      Austria   
..   ...           ...         ...             ...          ...   
15  1998        France      France          Brazil      Croatia   
16  2002   Korea/Japan      Brazil         Germany       Turkey   
17  2006       Germany       Italy          France      Germany   
18  2010  South Africa       Spain     Netherlands      Germany   
19  2014        Brazil     Germany       Argentina  Netherlands   

            Fourth  GoalsScored  QualifiedTeams  MatchesPlayed Attendance  \
0       Yugoslavia           70              13             18    590.549   
1          Austria           70              16             17    363.000   
2           Sweden           84              15             18    375.700   
3            Spain           88              13             22  1.045.246   
4          Uruguay          140              16             26    768.607   
..             ...          ...             ...            ...        ...   
15     Netherlands          171              32             64  2.785.100   
16  Korea Republic          161              32             64  2.705.197   
17        Portugal          147              32             64  3.359.439   
18         Uruguay          145              32             64  3.178.856   
19          Brazil          171              32             64  3.386.810   

    av_goals_matches  winner_and_hoster  brazil_winner  winner_uruguay  
0           3.888889               True            0.0               1  
1           4.117647               True            0.0               0  
2           4.666667              False            0.0               0  
3           4.000000              False            0.0               1  
4           5.384615              False            0.0               0  
..               ...                ...            ...             ...  
15          2.671875               True            0.0               0  
16          2.515625              False            1.0               0  
17          2.296875              False            0.0               0  
18          2.265625              False            0.0               0  
19          2.671875              False            0.0               0  

[20 rows x 14 columns]

#### Summary of `apply()` and `map()`

- `apply()` is used to apply a function along an axis of the DataFrame or on values of Series.
    - you are free to use lambda functions with map
    - you can also apply functions column-wise changing the axis=1 argument. 
    
- `map()` is used to substitute each value in a Series with another value.




### Recode using numpy

#### `np.where`: if-else approach.

- `np.where` is  similar to ifelse in R
- Useful if there’s only 1-2 (True/False conditions)
- sintax: `np.where(condition, true, false)`
- condition can be anything that returns a boolean. 

Let's see some examples:

In [132]:
# create a new variable
wc["winner_brazil"]=np.where(wc["Winner"]=="Brazil", 1, 0)
wc.head()

Year      Country      Winner      Runners-Up    Third      Fourth  \
0  1930      Uruguay     Uruguay       Argentina      USA  Yugoslavia   
1  1934        Italy       Italy  Czechoslovakia  Germany     Austria   
2  1938       France       Italy         Hungary   Brazil      Sweden   
3  1950       Brazil     Uruguay          Brazil   Sweden       Spain   
4  1954  Switzerland  Germany FR         Hungary  Austria     Uruguay   

   GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
0           70              13             18    590.549          3.888889   
1           70              16             17    363.000          4.117647   
2           84              15             18    375.700          4.666667   
3           88              13             22  1.045.246          4.000000   
4          140              16             26    768.607          5.384615   

   winner_and_hoster  brazil_winner  winner_uruguay  winner_brazil  
0               True            0.0               1              0  
1               True            0.0               0              0  
2              False            0.0               0              0  
3              False            0.0               1              0  
4              False            0.0               0              0

In [133]:
# notice we can easily use np.where with assign
wc.assign(winner_brazil_=np.where(wc["Winner"]=="Brazil", 1, 0)).head(10)

Year      Country      Winner      Runners-Up       Third        Fourth  \
0  1930      Uruguay     Uruguay       Argentina         USA    Yugoslavia   
1  1934        Italy       Italy  Czechoslovakia     Germany       Austria   
2  1938       France       Italy         Hungary      Brazil        Sweden   
3  1950       Brazil     Uruguay          Brazil      Sweden         Spain   
4  1954  Switzerland  Germany FR         Hungary     Austria       Uruguay   
5  1958       Sweden      Brazil          Sweden      France    Germany FR   
6  1962        Chile      Brazil  Czechoslovakia       Chile    Yugoslavia   
7  1966      England     England      Germany FR    Portugal  Soviet Union   
8  1970       Mexico      Brazil           Italy  Germany FR       Uruguay   
9  1974      Germany  Germany FR     Netherlands      Poland        Brazil   

   GoalsScored  QualifiedTeams  MatchesPlayed Attendance  av_goals_matches  \
0           70              13             18    590.549          3.888889   
1           70              16             17    363.000          4.117647   
2           84              15             18    375.700          4.666667   
3           88              13             22  1.045.246          4.000000   
4          140              16             26    768.607          5.384615   
5          126              16             35    819.810          3.600000   
6           89              16             32    893.172          2.781250   
7           89              16             32  1.563.135          2.781250   
8           95              16             32  1.603.975          2.968750   
9           97              16             38  1.865.753          2.552632   

   winner_and_hoster  brazil_winner  winner_uruguay  winner_brazil  \
0               True            0.0               1              0   
1               True            0.0               0              0   
2              False            0.0               0              0   
3              False            0.0               1              0   
4              False            0.0               0              0   
5              False            1.0               0              1   
6              False            1.0               0              1   
7               True            0.0               0              0   
8              False            1.0               0              1   
9              False            0.0               0              0   

   winner_brazil_  
0               0  
1               0  
2               0  
3               0  
4               0  
5               1  
6               1  
7               0  
8               1  
9               0

In [134]:
# using string methods
(wc
 .assign(south_america=np.where(wc["Winner"].str.contains("Brazil|Uruguay|Argentina"), 1, 0))
     .filter(["Winner", "south_america"])
     .head(10)
)

Winner  south_america
0     Uruguay              1
1       Italy              0
2       Italy              0
3     Uruguay              1
4  Germany FR              0
5      Brazil              1
6      Brazil              1
7     England              0
8      Brazil              1
9  Germany FR              0

#### `np.select`: for multiple conditions

- np.select: similar to `case_when` in R. 
- useful for when there’s multiple conditions to be recoded
- sintax: `np.select(conditon, choicelist, default)`

In [135]:
# step one: create a list of conditions
condition = [wc["Winner"]==wc["Country"], 
             wc["Runners-Up"]==wc["Country"],
             wc["Third"]==wc["Country"]
            ]
# step two: create the choice list
choice_list = [1, 2, 3]

# recode
(wc
    .assign(where_is_hoster=np.select(condition, choice_list, default="4+"))
    .filter(["where_is_hoster"])
)

where_is_hoster
0                1
1                1
2               4+
3                2
4               4+
..             ...
15               1
16              4+
17               3
18              4+
19              4+

[20 rows x 1 columns]

### Group by: split-apply-combine

**Functionality**:

- Grouping data by specific variables/column indices
- Summarize/aggregate data by specific group features

**How it works**

“group by” is a common data wrangling process that exists in any language (R, Python, SQL) and refers to a process involving one or more of the following steps:

- Splitting the data into groups based on some criteria.

- Applying a function to each group independently.

- Combining the results into a data structure.


![From Python Data Science Handbook by Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/figures/03.08-split-apply-combine.png)

### groupby (it just splits!)

The `groupby()` method in pandas splits your dataset in smaller parts. It generates an iterable where each group is broken up into a tuple (group,data). 

In [136]:
# load worldcup dataset
wc = pd.read_csv("WorldCups.csv")
wc_matches = pd.read_csv("WorldCupMatches.csv")

#groupby object
g = wc.groupby(["Winner"])
g

The output of a groupby method is a flexible abstraction: althugh more complicated things are happening under the hood, in many ways, it can be treated as a collection of ``DataFrame``s. And as we learned, any collector can be iterated over!


In [137]:
# Iteration for groups
for group, data in g:
    print(group)

Argentina
Brazil
England
France
Germany
Germany FR
Italy
Spain
Uruguay


/var/folders/jy/10_nyhkn3nv_rrbnd8f_fr940000gp/T/ipykernel_68851/2794934486.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group, data in g:


In [141]:
# iteration for data grouped
for group, data in g:
    print(data.head(2))

    Year    Country     Winner   Runners-Up   Third   Fourth  GoalsScored  \
10  1978  Argentina  Argentina  Netherlands  Brazil    Italy          102   
12  1986     Mexico  Argentina   Germany FR  France  Belgium          132   

    QualifiedTeams  MatchesPlayed Attendance  
10              16             38  1.545.791  
12              24             52  2.394.031  
   Year Country  Winner      Runners-Up   Third      Fourth  GoalsScored  \
5  1958  Sweden  Brazil          Sweden  France  Germany FR          126   
6  1962   Chile  Brazil  Czechoslovakia   Chile  Yugoslavia           89   

   QualifiedTeams  MatchesPlayed Attendance  
5              16             35    819.810  
6              16             32    893.172  
   Year  Country   Winner  Runners-Up     Third        Fourth  GoalsScored  \
7  1966  England  England  Germany FR  Portugal  Soviet Union           89   

   QualifiedTeams  MatchesPlayed Attendance  
7              16             32  1.563.135  
    Year Co

/var/folders/jy/10_nyhkn3nv_rrbnd8f_fr940000gp/T/ipykernel_68851/4250474981.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group, data in g:


And we can acess a specific group:

In [142]:
g.get_group("Argentina")

Year    Country     Winner   Runners-Up   Third   Fourth  GoalsScored  \
10  1978  Argentina  Argentina  Netherlands  Brazil    Italy          102   
12  1986     Mexico  Argentina   Germany FR  France  Belgium          132   

    QualifiedTeams  MatchesPlayed Attendance  
10              16             38  1.545.791  
12              24             52  2.394.031

### Aggreations (summarize)

The power of a grouping function (like `.groupby()` shines when coupled with an aggregation operation.

An aggregation is any operation that reduces the dimensionality of the data!

#### `pandas`: `.groupby()` +  built-in methods

In [143]:
# mean of all numeric variables grouping by winners
wc.groupby(["Winner"]).mean()

/var/folders/jy/10_nyhkn3nv_rrbnd8f_fr940000gp/T/ipykernel_68851/2212436017.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  wc.groupby(["Winner"]).mean()


Year  GoalsScored  QualifiedTeams  MatchesPlayed
Winner                                                             
Argentina   1982.000000   117.000000       20.000000      45.000000
Brazil      1977.200000   122.400000       20.800000      43.000000
England     1966.000000    89.000000       16.000000      32.000000
France      1998.000000   171.000000       32.000000      64.000000
Germany     2014.000000   171.000000       32.000000      64.000000
Germany FR  1972.666667   117.333333       18.666667      38.666667
Italy       1965.000000   111.750000       21.750000      37.750000
Spain       2010.000000   145.000000       32.000000      64.000000
Uruguay     1940.000000    79.000000       13.000000      20.000000

or select a specific variable to perform the aggregation step on. 

In [144]:
# With a specific input
wc.groupby(["Winner"])["GoalsScored"].mean()

Winner
Argentina     117.000000
Brazil        122.400000
England        89.000000
France        171.000000
Germany       171.000000
Germany FR    117.333333
Italy         111.750000
Spain         145.000000
Uruguay        79.000000
Name: GoalsScored, dtype: float64

Notice the results come in a index structure. You can easily convert back to a fully formated dataframe by: 

In [145]:
# reseting the index
wc.groupby(["Winner"])["GoalsScored"].mean().reset_index()

Winner  GoalsScored
0   Argentina   117.000000
1      Brazil   122.400000
2     England    89.000000
3      France   171.000000
4     Germany   171.000000
5  Germany FR   117.333333
6       Italy   111.750000
7       Spain   145.000000
8     Uruguay    79.000000

In [146]:
## as_index=false
wc.groupby(["Winner"], as_index=False)["GoalsScored"].mean()

Winner  GoalsScored
0   Argentina   117.000000
1      Brazil   122.400000
2     England    89.000000
3      France   171.000000
4     Germany   171.000000
5  Germany FR   117.333333
6       Italy   111.750000
7       Spain   145.000000
8     Uruguay    79.000000

Pandas offers many built-in methods to perform aggregations. Here is a list:

#### Built-in Methods

[See user guide from Pandas Documentation](https://pandas.pydata.org/docs/user_guide/groupby.html)

| Method  | Functionality |
|:---------------:|:-----------------------------|
|`.any`|Compute whether any of the values in the groups are truthy| 
|`.all`|Compute whether all of the values in the groups are truthy|
|`.count`|Compute the number of non-NA values in the groups|
|`.cov` |Compute the covariance of the groups|
|`.first`|Compute the first occurring value in each group|
|`.idxmax` |Compute the index of the maximum value in each group|
| `.idxmin`| Compute the index of the minimum value in each group|
|`.last`|Compute the last occurring value in each group|
|`.max`|Compute the maximum value in each group|
|`.mean`|Compute the mean of each group|
|`.median`|Compute the median of each group|
|`.min`|Compute the minimum value in each group|
|`.nunique`|Compute the number of unique values in each group|
|`.prod`|Compute the product of the values in each group|
|`.quantile`|Compute a given quantile of the values in each group|
|`.sem`|Compute the standard error of the mean of the values in each group|
|`.size`|Compute the number of values in each group|
|`.skew` |Compute the skew of the values in each group|
|`.std`|Compute the standard deviation of the values in each group|
|`.sum`|Compute the sum of the values in each group|
|`.var`|Compute the variance of the values in each group|

Let's see some examples:

In [147]:
# max goal by team as home team
(wc_matches
     .groupby(["Home Team Name"])
     ["Home Team Goals"]
     .max()
     .reset_index()
     .sort_values("Home Team Goals", ascending=False))

Home Team Name  Home Team Goals
31                   Hungary               10
71                Yugoslavia                9
69                   Uruguay                8
62                    Sweden                8
25                   Germany                8
..                       ...              ...
56                    Serbia                0
17            Czech Republic                0
64                      Togo                0
32                   IR Iran                0
77  rn">United Arab Emirates                0

[78 rows x 2 columns]

In [148]:
# number of matches as home team
# max goal by team as home team
(wc_matches
     .groupby(["Home Team Name"], as_index=False)
     ["Home Team Name"]
     .size()
     .sort_values("size", ascending=False)
)

Home Team Name  size
7                     Brazil    82
35                     Italy    57
2                  Argentina    54
26                Germany FR    43
22                   England    35
..                       ...   ...
32                   IR Iran     1
43               New Zealand     1
33                      Iran     1
1                     Angola     1
77  rn">United Arab Emirates     1

[78 rows x 2 columns]

#### `pandas`: `.groupby()` + `agg()`

Alternatively, we can specify a whole range of operations to aggregate by (along with specific variable columns) using the `.aggregate()`/`.agg()` method. To keep track of which operations correspond which variable, `pandas` will generate a hierarchical index for column entries. 

In [149]:
wc.groupby(["Winner"])["GoalsScored"].agg(["mean","std","median"])

mean        std  median
Winner                                   
Argentina   117.000000  21.213203   117.0
Brazil      122.400000  30.476220   126.0
England      89.000000        NaN    89.0
France      171.000000        NaN   171.0
Germany     171.000000        NaN   171.0
Germany FR  117.333333  21.594752   115.0
Italy       111.750000  40.532908   115.0
Spain       145.000000        NaN   145.0
Uruguay      79.000000  12.727922    79.0

We can also **_user-defined functions_**  into the `aggregate()` function as well.

In [151]:
def mean_add_50(x):
    return np.mean(x) + 50

wc.groupby(["Winner"])["GoalsScored"].agg(["mean","std","median",mean_add_50])

mean        std  median  mean_add_50
Winner                                                
Argentina   117.000000  21.213203   117.0   167.000000
Brazil      122.400000  30.476220   126.0   172.400000
England      89.000000        NaN    89.0   139.000000
France      171.000000        NaN   171.0   221.000000
Germany     171.000000        NaN   171.0   221.000000
Germany FR  117.333333  21.594752   115.0   167.333333
Italy       111.750000  40.532908   115.0   161.750000
Spain       145.000000        NaN   145.0   195.000000
Uruguay      79.000000  12.727922    79.0   129.000000

`agg()` + `.rename()` provides an easy workflow to rename your newly created variables

In [152]:
(wc.groupby(["Winner"])["GoalsScored"].
    agg(["mean","std","median",mean_add_50]).
    rename(columns={"mean": "goals_mean", 
                    "std": "goals_std",
                    "median": "goals_median", 
                    "mean_add_50":"mean_50_goals"})
)   

goals_mean  goals_std  goals_median  mean_50_goals
Winner                                                        
Argentina   117.000000  21.213203         117.0     167.000000
Brazil      122.400000  30.476220         126.0     172.400000
England      89.000000        NaN          89.0     139.000000
France      171.000000        NaN         171.0     221.000000
Germany     171.000000        NaN         171.0     221.000000
Germany FR  117.333333  21.594752         115.0     167.333333
Italy       111.750000  40.532908         115.0     161.750000
Spain       145.000000        NaN         145.0     195.000000
Uruguay      79.000000  12.727922          79.0     129.000000

####  `pandas`: `.groupby()` + `.apply()` 

Even though I cover this in more details on the miscellaneous notebook, here is a good moment to bring the `.apply()` methods from pandas one more time. 

The `apply` method lets you apply an arbitrary function by row or by columns on your data frame. As you can imagine, it also allow you to apply results by at a `groupby` object, and returns a summarizes dataset. 

The function should take a `DataFrame` and returns either a Pandas object (e.g., `DataFrame`, `Series`) or a scalar

Let's an example with the mean add function:

In [22]:
(wc.groupby(["Country"])["GoalsScored"]. 
     apply(mean_add_50). 
     reset_index()
)

Country  GoalsScored
0      Argentina        152.0
1         Brazil        179.5
2          Chile        139.0
3        England        139.0
4         France        177.5
5        Germany        172.0
6          Italy        142.5
7    Korea/Japan        211.0
8         Mexico        163.5
9   South Africa        195.0
10         Spain        196.0
11        Sweden        176.0
12   Switzerland        190.0
13           USA        191.0
14       Uruguay        120.0

#### multi-index grouping

We can also group by more than one variable (i.e. implement a **_multi-index on the rows_**).

In [153]:
wc.groupby(["Winner", "Year"])["GoalsScored"].mean().reset_index().head(5)

Winner  Year  GoalsScored
0  Argentina  1978        102.0
1  Argentina  1986        132.0
2     Brazil  1958        126.0
3     Brazil  1962         89.0
4     Brazil  1970         95.0

#### `pandas`: `.groupby()` +   `.transform()`

Other times we want to implement data manipulations by some grouping variable but retain structure of the original data. Put differently, our aim is not to aggregate but to perform some operation across specific groups. 

To do so, we will combine `.groupby()` and `.transform()` methods. Let's see an example: 

In [154]:
# create a new column
# notive you need to select the column you want to transform. 
wc.groupby(["Winner"])["GoalsScored"].transform("mean").head(5)

0     79.000000
1    111.750000
2    111.750000
3     79.000000
4    117.333333
Name: GoalsScored, dtype: float64

In [155]:
# easily combined with assign
# create a new column
(wc.assign(goals_score_wc_mean_wc=wc.groupby(["Winner"])["GoalsScored"].
    transform("mean"))).head()

Year      Country      Winner      Runners-Up    Third      Fourth  \
0  1930      Uruguay     Uruguay       Argentina      USA  Yugoslavia   
1  1934        Italy       Italy  Czechoslovakia  Germany     Austria   
2  1938       France       Italy         Hungary   Brazil      Sweden   
3  1950       Brazil     Uruguay          Brazil   Sweden       Spain   
4  1954  Switzerland  Germany FR         Hungary  Austria     Uruguay   

   GoalsScored  QualifiedTeams  MatchesPlayed Attendance  \
0           70              13             18    590.549   
1           70              16             17    363.000   
2           84              15             18    375.700   
3           88              13             22  1.045.246   
4          140              16             26    768.607   

   goals_score_wc_mean_wc  
0               79.000000  
1              111.750000  
2              111.750000  
3               79.000000  
4              117.333333

In [156]:
# also: very useful with lambda functions
wc.groupby("Winner")["GoalsScored"].transform(lambda x: x - x.mean())

0     -9.000000
1    -41.750000
2    -27.750000
3      9.000000
4     22.666667
        ...    
15     0.000000
16    38.600000
17    35.250000
18     0.000000
19     0.000000
Name: GoalsScored, Length: 20, dtype: float64

### Sorting values

In [157]:
# Pandas: sort values by a column variable (ascending)
wc.sort_values('Country').head(3)

Year    Country     Winner   Runners-Up        Third  Fourth  GoalsScored  \
10  1978  Argentina  Argentina  Netherlands       Brazil   Italy          102   
19  2014     Brazil    Germany    Argentina  Netherlands  Brazil          171   
3   1950     Brazil    Uruguay       Brazil       Sweden   Spain           88   

    QualifiedTeams  MatchesPlayed Attendance  
10              16             38  1.545.791  
19              32             64  3.386.810  
3               13             22  1.045.246

In [158]:
# Pandas: sort values by a column variable (descending)
wc.sort_values('Year',ascending=False).head(3)

Year       Country   Winner   Runners-Up        Third    Fourth  \
19  2014        Brazil  Germany    Argentina  Netherlands    Brazil   
18  2010  South Africa    Spain  Netherlands      Germany   Uruguay   
17  2006       Germany    Italy       France      Germany  Portugal   

    GoalsScored  QualifiedTeams  MatchesPlayed Attendance  
19          171              32             64  3.386.810  
18          145              32             64  3.178.856  
17          147              32             64  3.359.439

In [159]:
# Pandas: sort values by more than one column variable 
wc.sort_values(['Winner', "Country"]).head(3)

Year    Country     Winner      Runners-Up   Third      Fourth  \
10  1978  Argentina  Argentina     Netherlands  Brazil       Italy   
12  1986     Mexico  Argentina      Germany FR  France     Belgium   
6   1962      Chile     Brazil  Czechoslovakia   Chile  Yugoslavia   

    GoalsScored  QualifiedTeams  MatchesPlayed Attendance  
10          102              16             38  1.545.791  
12          132              24             52  2.394.031  
6            89              16             32    893.172

### That was a lot! but you will get to keep this notebook for you! 

And remember of the [cheat sheet for pandas](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

## Practice 

Using the same ACLED data from the previous exercise, answer:

1. What are the different event types recorded?

2. How many events are recorded for each year?

3. What’s the most common event type in the data?

4. Which countries had the highest number of reported fatalities?

In [ ]:
## Add your code here